# Imports and meshcat

In [1]:
from functools import partial
import numpy as np
from pydrake.all import (
    AddUnitQuaternionConstraintOnPlant,
    AutoDiffXd,
    ExtractGradient,
    AddDefaultVisualization,
    MathematicalProgram,
    OrientationConstraint,
    PiecewisePolynomial,
    PositionConstraint,
    RotationMatrix,
    Simulator,
    SnoptSolver,
    Solve,
    StartMeshcat,
    eq,
    namedview,
    DiscreteContactApproximation,
    RobotDiagramBuilder,
    VectorLogSink
)

from utils.utils_urdf import ConfigureParser, load_urdf_file

In [2]:
# Start the visualizer (run this cell only once, each instance consumes a port)
meshcat = StartMeshcat()

INFO:drake:Meshcat listening for connections at http://localhost:7001


# Utility functions for custom constraints

- one for equality operator to compare numbers with derivates
- one for **velocity**,  -> $(q[n+1] - q[n])/h = v[n]$
- one for **angular momentum** -> $I\ddot{\theta} = \sum p_i \times f_i$
- one for **Center of Mass** -> make sure com and H of `prog` coincide with context
- one for **foot in contact**, i.e., foot is not moving/sliding during stance
    

here is a template for custom constraint functions:

```python
def constraint_fun(vars, plant, plant_ad, context, context_ad, time_index):
    # extract relevant variables
    x,y,z = vars.split(n_x, n_x+n_y)
    # check if derivative is demanded
    if isInstance(vars[0], AutoDiffXd):
        # use context with autodiff
        # check if the context is up to date:
        if not AD_equal(x, context_ad.get_x()):
            plant_ad.set_x(context_ad[time_ind], x)
        # use plant kinematics to compute relevant quantity:
        y_context = plant_ad.compute_y(context_ad[time_ind])
    # if derivative is not demanded do everythign with usual context
    else:
        # check if the context is up to date:
        if not AD_equal(x, context.get_x()):
            plant.set_x(context[time_ind], x)
        # use plant kinematics to compute relevant quantity:
        y_context = plant.compute_y(context[time_ind])
    
    # return error
    return y = y_context

### 0. Equality operator for AutoDiff

we need to write equal operator to check equality for variables with autoDiff property enabled

In [3]:
def AD_equal(a, b):
    return np.array_equal(a, b) and np.array_equal(
        ExtractGradient(a), ExtractGradient(b)
    )

### 1.Velocity
We use `plant.MapQDotToVelocity` to convert qdot into generalized velocity, i.e., from quaternion to xyz velocity

In [4]:
def velocity_matching_constraints(vars, plant, plant_ad, context, context_ad, time_index):
    # extract variables
    n_q, n_v = plant.num_positions(), plant.num_velocities()
    h, q, v, qn = np.split(vars, [1,1+n_q, 1+n_q+n_v])
    
    # if demands derivatives
    if isinstance(vars[0], AutoDiffXd):
        # check if up to date
        if not AD_equal(q, plant_ad.GetPositions(context_ad[time_index])):
            plant_ad.SetPositions(context_ad[time_index], q)
        # get velocity from context 
        v_context = plant_ad.MapQDotToVelocity(context_ad[time_index], (qn - q)/h)
    else:
        # check if up to date
        if not np.array_equal(q, plant.GetPositions(context[time_index])):
            plant.SetPositions(context[time_index])
        # get veloctiy
        v_context = plant.MapQDotToVelocity(context[time_index], (qn - q)/h)

    # return error
    return v  - v_context

### 2. Momentum Matching

* `plant.CalcSpatialMomentumInWorldAbout().rotational()`: compute spatial momentum and extract rotational part 
* `plant.CalcCenterOfMAssPositionInWorld()`: compute the center of mass (depending on configuration)

In [5]:
def momentum_matching_constraints(vars, plant, plant_ad, context, context_ad, time_index, model):
    n_qv = plant.num_multibody_states()
    qv, com, H = np.split(vars, [n_qv,n_qv+3])
    # if demands derivatives
    if isinstance(vars[0], AutoDiffXd):
        # check if up to date
        if not AD_equal(qv, plant_ad.GetPositionsAndVelocities(context_ad[time_index])):
            plant_ad.SetPositionsAndVelocities(context_ad[time_index], qv)
        # get com and momentum from context 
        com_context = plant_ad.CalcCenterOfMassPositionInWorld(context_ad[time_index], [model])
        # TODO: com_context or com
        H_context = plant_ad.CalcSpatialMomentumInWorldAboutPoint(
                                    context_ad[time_index], 
                                    [model], 
                                    com_context
                                ).rotational()
    else:
        # check if up to date
        if not np.array_equal(qv, plant.GetPositionsAndVelocities(context[time_index])):
            plant.SetPositionsAndVelocities(context[time_index], qv)
        # get com and H
        com_context = plant.CalcCenterOfMassPositionInWorld(context[time_index], [model])
        H_context = plant.CalcSpatialMomentumInWorldAboutPoint(
                                    context[time_index], 
                                    [model], 
                                    com_context
                                ).rotational()
    
    # return error
    return np.concatenate([com - com_context, H - H_context])

### 3. Angular Momentum

- we use `CalcPointPositions` for finding position of the contact (origin in foot frame)
- $\dot{H} = \sum_i p_i \times F_i$

In [6]:
def angular_momentum_constraints(vars, plant, plant_ad, context, context_ad, time_index, foot_frames, foot_frames_ad, p_foot):
    n_q = plant.num_positions()
    q, com, Hdot, F = np.split(vars, [n_q,n_q+3, n_q+6])
    F = F.reshape(3,4,order='F')
    
    # if demands derivatives
    if isinstance(vars[0], AutoDiffXd):
        # check if up to date
        if not AD_equal(q, plant_ad.GetPositions(context_ad[time_index])):
            plant_ad.SetPositions(context_ad[time_index], q)
        # compute total torque on com from each foot
        torque = np.zeros(3, dtype=AutoDiffXd)
        for i in range(4):
            p_WF = plant_ad.CalcPointsPositions(
                    context_ad[time_index], 
                    foot_frames_ad[i],
                    p_foot,
                    plant_ad.world_frame(),
                )
            torque += np.cross(p_WF.flatten() - com, F[:,i])

            
    else:
        ## check if up to date
        if not np.array_equal(q, plant.GetPositions(context[time_index])):
            plant.SetPositions(context[time_index], q)
        # compute total torque on com from each foot
        torque = np.zeros(3)
        for i in range(4):
            p_WF = plant.CalcPointPositions(
                    context[time_index], 
                    foot_frames[i],
                    p_foot,
                    plant.world_frame(),
                )
            torque += np.cross(p_WF.flatten() - com, F[:,i])
    
    # return error
    return Hdot - torque

### 4. Contact constraints
- use `CalcPointPositions` to locate end effector
- compute the difference between contact points at two consecutive moment

In [7]:
def contact_constraints(vars, plant, plant_ad, context, context_ad, time_index, frame, frame_ad, p_foot):
    n_q = plant.num_positions()
    q, qn = np.split(vars, [n_q])

    # if demands derivatives
    if isinstance(vars[0], AutoDiffXd):
        # check if up to date
        if not AD_equal(q, plant_ad.GetPositions(context_ad[time_index])):
            plant_ad.SetPositions(context_ad[time_index], q)
        if not AD_equal(qn, plant_ad.GetPositions(context_ad[time_index+1])):
            plant_ad.SetPositions(context_ad[time_index+1], qn)

        # compute position of foot
        p_WF = plant_ad.CalcPointsPositions(
                    context_ad[time_index], 
                    frame_ad, 
                    p_foot,
                    plant_ad.world_frame(),
                )
        
        p_WF_n = plant_ad.CalcPointsPositions(
                    context_ad[time_index+1], 
                    frame_ad, 
                    p_foot,
                    plant_ad.world_frame(),
                )
            
    else:
        ## check if up to date
        if not np.array_equal(q, plant.GetPositions(context[time_index])):
            plant.SetPositions(context[time_index], q)
        if not np.array_equal(qn, plant.GetPositions(context[time_index+1])):
            plant.SetPositions(context[time_index+1], qn)
            
        # compute position of foot
        p_WF = plant.CalcPointsPositions(
                    context[time_index], 
                    frame, 
                    p_foot,
                    plant.world_frame(),
                )
        
        p_WF_n = plant.CalcPointsPositions(
                    context[time_index+1], 
                    frame, 
                    p_foot,
                    plant.world_frame(),
                )
    
    # return error
    return p_WF_n - p_WF

# Configure the plant
let's summon the Spot!

In [8]:
robot_builder = RobotDiagramBuilder(time_step=1e-3)
builder = robot_builder.builder()
parser = robot_builder.parser()
plant = robot_builder.plant()
ConfigureParser(parser)
parser.AddModelsFromString(load_urdf_file('models/spot.urdf'),".urdf" )
parser.AddModelsFromString(load_urdf_file('models/ground_mit.urdf'),".urdf" )
plant.set_discrete_contact_approximation(DiscreteContactApproximation.kLagged)
plant.Finalize()

model = plant.GetModelInstanceByName("spot")

## Visualization and loggers

In [9]:
# visuailize: meshcat
AddDefaultVisualization(builder, meshcat=meshcat)

# logger
state_logger = builder.AddSystem(VectorLogSink(plant.num_multibody_states()))
actuation_logger = builder.AddSystem(VectorLogSink(plant.num_actuators()))
builder.Connect(plant.get_state_output_port(), state_logger.get_input_port())
builder.Connect(plant.get_net_actuation_output_port(), actuation_logger.get_input_port())

# finalize
diagram = builder.Build()

LCM self test failed!!
Check your routing tables and firewall settings


## Print body parts, frames, names etc

In [10]:
print('=====++++++++++++++== body names ===============')
for i in plant.GetBodyIndices(model):
    print(plant.get_body(i))
print('=====================  state names ===============')
for i in plant.GetStateNames(False):
    print(i)
print('===================== actuation names ===============')
for i in plant.GetActuatorNames(model, False):
    print(i)
print('===================== frame names ===============')
for i in plant.GetFrameIndices(model):
    print(plant.get_frame(i))
print('===================== default positions ===============')
print(plant.GetDefaultPositions())

=====++++++++++++++== body names ===============
<RigidBody name='body' index=1 model_instance=2>
<RigidBody name='base_link' index=2 model_instance=2>
<RigidBody name='front_rail' index=3 model_instance=2>
<RigidBody name='rear_rail' index=4 model_instance=2>
<RigidBody name='front_left_hip' index=5 model_instance=2>
<RigidBody name='front_left_upper_leg' index=6 model_instance=2>
<RigidBody name='front_left_lower_leg' index=7 model_instance=2>
<RigidBody name='front_right_hip' index=8 model_instance=2>
<RigidBody name='front_right_upper_leg' index=9 model_instance=2>
<RigidBody name='front_right_lower_leg' index=10 model_instance=2>
<RigidBody name='rear_left_hip' index=11 model_instance=2>
<RigidBody name='rear_left_upper_leg' index=12 model_instance=2>
<RigidBody name='rear_left_lower_leg' index=13 model_instance=2>
<RigidBody name='rear_right_hip' index=14 model_instance=2>
<RigidBody name='rear_right_upper_leg' index=15 model_instance=2>
<RigidBody name='rear_right_lower_leg' ind

## Simulation

In [11]:
# simulation
simulator = Simulator(diagram)
context = diagram.CreateDefaultContext()
plant_context = plant.GetMyContextFromRoot(context)

# desired_state
# desired_angles = np.zeros(controller.get_input_port_desired_state().size())
# desired_angles[:12] = plant.GetDefaultPositions()[7:19]
# desired_angles = plant.get_state_output_port().Eval(plant_context)

simulator.reset_context(context)
simulator.set_target_realtime_rate(0.0)
meshcat.StartRecording()
simulator.AdvanceTo(3.0)
meshcat.PublishRecording()

## Plant Parameters

In [12]:
# state size
n_q = plant.num_positions()
n_v = plant.num_velocities()


mu = 1  # rubber on rubber
total_mass = plant.CalcTotalMass(plant_context, [model])
gravity = plant.gravity_field().gravity_vector()
foot_frame = [
    plant.GetFrameByName("front_left_lower_leg"),
    plant.GetFrameByName("front_right_lower_leg"),
    plant.GetFrameByName("rear_left_lower_leg"),
    plant.GetFrameByName("rear_right_lower_leg"),
]
body_frame = plant.GetFrameByName('body')
plant_ad = plant.ToAutoDiffXd()
foot_frame_ad = [
    plant_ad.GetFrameByName("front_left_lower_leg"),
    plant_ad.GetFrameByName("front_right_lower_leg"),
    plant_ad.GetFrameByName("rear_left_lower_leg"),
    plant_ad.GetFrameByName("rear_right_lower_leg"),
]


### Named reference

In [13]:
PositionView = namedview(
    "Positions", plant.GetPositionNames(model, always_add_suffix=False)
)
VelocityView = namedview(
    "Velocities",
    plant.GetVelocityNames(model, always_add_suffix=False),
)


the following is the position of foot in each foot (in the foot frame). I could not find a way to extract this using Drake. For now, I check URDF file and collision geometry at lower leg

In [14]:
p_FootFootTip = np.array([0.0, 0.0, -0.3725])

## Gait schedule
gait schedule and relevant variables such as number of breakpoints used in one step, stride length, speed etc.

In [15]:
# walking
# num of breakpoints
N = 42
in_stance = np.zeros((4, N))
in_stance = np.zeros((4, N))
in_stance[0, :11] = 1
in_stance[0, 29:] = 1
in_stance[1, 8:21] = 1
in_stance[1, 21:32] = 1
in_stance[2, 8:21] = 1
in_stance[2, 21:32] = 1
in_stance[3, :11] = 1
in_stance[3, 29:] = 1
speed = 0.4
stride_length = 0.25

# gait parameters
one_step = stride_length
T = one_step / speed
min_clearence = 0.01 #min height of the foot at air

# Hybrid trajectory optimization
we start with creating an instance of mathematical program

In [19]:
prog = MathematicalProgram()

and the optimization variables to be used in trajectory optimization

In [20]:
# step size (time intervals between breakpoints)
h = prog.NewContinuousVariables(N-1, "h")

# position and velocity
q = prog.NewContinuousVariables(n_q, N, "q")
v = prog.NewContinuousVariables(n_v, N, 'v')

# contact forces for 4 feet
contact_forces = [prog.NewContinuousVariables(3,N,f'foot{i}_contact_force') for i in range(4)]

# Center of Mass (CoM) variables and derivatives for ZMP like implementation
com = prog.NewContinuousVariables(3, N, 'com')
comdot = prog.NewContinuousVariables(3, N, 'comdot')
comddot = prog.NewContinuousVariables(3, N-1, 'comddot')

# angular momentum and derivative
H = prog.NewContinuousVariables(3, N,"H")
Hdot = prog.NewContinuousVariables(3, N,'H')

Initially, the decision variables in a `MathematicalProgram` (like position `q` and velocity `v`) are just abstract numbers. They have no inherent connection to physical laws or kinematic limits.

We connect these variables to the robot by comparing them with what **context** would provide. In this way, we enforce the robot's physical model (`MultibodyPlant`)and the rules of physics.

For example, the optimizer is free to choose any values for the position variables `q_n` (at time `n`) and `q_{n+1}` (at time `n+1`). However, a dynamics constraint is then added to ensure this change in position is physically consistent with the velocity `v_n`. In turn, other constraints ensure that `q` and `v` respect the robot's physical joint limits.

We create a context for each breakpoint to increase number of cache-hits. For example, when we want to compute the position of end effector at a position, we do forward kinematics using context which stores the result of inquiry in the cache. Later, when the `prog` demands the same info, instead of computing the same forward kinematics, the context provides the answer from the cache. 

Another aspect of context is to be able to use built-in **AutoDiff**. We create an instance of the plant that enables **AutoDiff** so that the associated context keeps the derivative info as well. The derivatives are used when the optimizer tries to improve constraints. Generally, we use context with autodiff when we write custom constraints. 


### Context definitions

In [21]:
context = [plant.CreateDefaultContext() for i in range(N)]
context_ad_velocity = [plant_ad.CreateDefaultContext() for i in range(N)]
context_ad_foot_position = [plant_ad.CreateDefaultContext() for i in range(N)]
context_ad_com_momentum = [plant_ad.CreateDefaultContext() for i in range(N)]
context_ad_contact = [plant_ad.CreateDefaultContext() for i in range(N)]

## Constriants
### 1. Constraints on the time step `h`
- $h_l \leq h_i \leq h_u$
- $(1-c)T \leq \sum_i h_i \leq (1+c)T$ for some $c$

In [22]:
prog.AddBoundingBoxConstraint(0.5 * T / N, 2 * T / N, h)
prog.AddLinearConstraint(0.9 * T <= sum(h))
prog.AddLinearConstraint(1.1 * T >= sum(h))

### 2. Joint constraints
- $q_l \leq q\leq q_u$
- $v_l \leq v \leq v_u$

In [23]:
for n in range(N):
    prog.AddBoundingBoxConstraint(
        plant.GetPositionLowerLimits(), 
        plant.GetPositionUpperLimits(),
        q[:, n]
    )

    prog.AddBoundingBoxConstraint(
        plant.GetVelocityLowerLimits(),
        plant.GetVelocityUpperLimits(),
        v[:, n]
    )


### 3. Quaternion constraints
- $q_w^2 + q_x^2 + q_y^2 + q_z^2 = 1$

In [24]:
for n in range(N):
    AddUnitQuaternionConstraintOnPlant(plant, q[:, n], prog)
    

### 4. Body orientation
body should be approximately parallel to the ground (or in upward position for humanoids)
- drake handles this using `OrientationConstraint`
- we just provide, the plant, frame of the body part and its rotation matrix (identity of there is none), world frame and its rotation matrix, allowable rotation angle in any axis (in radian), context

In [25]:
for n in range(N):
    prog.AddConstraint(
        OrientationConstraint(
            plant, 
            body_frame, 
            RotationMatrix(), 
            plant.world_frame(), 
            RotationMatrix(), 
            3/180*np.pi, 
            context[n]
        ),
        q[:,n]
    )

### 5. Velocity matching constraints
- `v` variable should match velocity of the `Plant`
- use our custom function

In [26]:
for n in range(N-1):
    custom_constraint = partial(
        velocity_matching_constraints, 
        plant=plant, 
        plant_ad=plant_ad, 
        context=context,
        context_ad=context_ad_velocity,
        time_index=n,
    )
    prog.AddConstraint(
        custom_constraint,
        lb=np.zeros(n_v),
        ub=np.zeros(n_v),
        vars=np.concatenate([[h[n]], q[:,n], v[:,n], q[:,n+1]]),
    )

### 6. CoM and AngMom Matching constraints
- center of mass `com` and angular momentum `H` should match those coming from `plant`
- use our custom function

In [27]:
for n in range(N):
    custom_constraint = partial(
        momentum_matching_constraints, 
        plant=plant, 
        plant_ad=plant_ad, 
        context=context,
        context_ad=context_ad_com_momentum,
        time_index=n,
        model=model
    )
    prog.AddConstraint(
        custom_constraint,
        lb=np.zeros(6),
        ub=np.zeros(6),
        vars=np.concatenate([q[:,n], v[:,n], com[:,n], H[:,n]]),
    )

### 7. Angular Momentum Constraints
- total angular momentum == total torque at the center of mass
- $I\dot{\theta} = \sum_i p_i \times f_i$
- use custom function

In [28]:
for n in range(N):
    custom_constraint = partial(
        angular_momentum_constraints, 
        plant=plant, 
        plant_ad=plant_ad, 
        context=context,
        context_ad=context_ad_foot_position,
        time_index=n,
        foot_frames=foot_frame,
        foot_frames_ad=foot_frame_ad,
        p_foot=p_FootFootTip
    )
    Fn = np.concatenate([contact_forces[i][:, n] for i in range(4)])
    prog.AddConstraint(
        custom_constraint,
        lb=np.zeros(3),
        ub=np.zeros(3),
        vars=np.concatenate([q[:,n], com[:,n], Hdot[:,n], Fn]),
    )

### 8. Linearized friction cone
- $|f_x|\leq \mu f_z$
- $0 \leq f_z \leq mg$ if no stance else zero

In [29]:
for n in range(N):
    for foot in range(4):
        prog.AddLinearConstraint(contact_forces[foot][0,n] <= mu * contact_forces[foot][2,n])
        prog.AddLinearConstraint(-contact_forces[foot][0,n] <= mu * contact_forces[foot][2,n])
        prog.AddLinearConstraint(contact_forces[foot][1,n] <= mu * contact_forces[foot][2,n])
        prog.AddLinearConstraint(-contact_forces[foot][1,n] <= mu * contact_forces[foot][2,n])
        prog.AddLinearConstraint(0 <= contact_forces[foot][2,n])
        prog.AddLinearConstraint(
            contact_forces[foot][2,n] <= in_stance[foot, n] * 10 * total_mass * np.abs(gravity[-1])
        )

### 9. Dynamics
dynamics of
- CoM
- angular momentum
- right using forward eular
- we could use implicit instead of explicit Euler

In [30]:
for n in range(N-1):
    # comdot
    prog.AddConstraint(eq(com[:,n+1], com[:,n] + h[n]*comdot[:,n]))
    
    # comddot
    prog.AddConstraint(eq(comdot[:,n+1], comdot[:,n] + h[n]*comddot[:,n]))
    
    # angular momentum
    prog.AddConstraint(eq(H[:,n+1],  H[:,n] + h[n]*Hdot[:,n]))

- $m\ddot{x}_{cm} == F + mg$

In [31]:
for n in range(N-1):
    prog.AddConstraint(
                eq(total_mass * comddot[:, n], 
                    sum([contact_forces[i][:, n] for i in range(4)]) + total_mass * gravity
                )
            )

### 10. Center of Mass trajectory constraints 
Let $x,y,z$ be the coordinates of CoM:
- $x[0] = y[0] = 0$
- $\dot{z}[0] = 0$
- $z \geq 0.125$
- $\dot{x} \geq 0$
- $x[N] = $ step length 

In [32]:
prog.AddBoundingBoxConstraint(0,0,com[:2,0])
prog.AddBoundingBoxConstraint(0,0,comdot[2,0])
prog.AddBoundingBoxConstraint(0.125, np.inf, com[2,:])
prog.AddBoundingBoxConstraint(0, np.inf, comdot[0,:])
prog.AddBoundingBoxConstraint(one_step, one_step, com[0,-1])

### 11. Foot positions (moving or in stance)
- if `in_stance` == True, foot should be on ground (z ==0 in world frame)
- if `in_stance` and `n>0`, position should not change
- we use custom function for this constraint
- else we have a minimum clearance, i.e., z>0.1

In [33]:
for i in range(4):
    for n in range(N):
        if in_stance[i,n]:
            # check if z==0
            prog.AddConstraint(
                PositionConstraint(
                    plant,
                    plant.world_frame(),
                    [-np.inf, -np.inf, 0],
                    [np.inf, np.inf, 0],
                    foot_frame[i],
                    p_FootFootTip,
                    context[n]
                ),
                vars=q[:,n]
            )
            # do not move during stance
            if n > 0 and in_stance[i,n-1]:
                custom_constraint = partial(
                    contact_constraints, 
                    plant=plant, 
                    plant_ad=plant_ad, 
                    context=context,
                    context_ad=context_ad_contact,
                    time_index=n-1,
                    frame=foot_frame[i],
                    frame_ad=foot_frame_ad[i],
                    p_foot=p_FootFootTip
                )
                prog.AddConstraint(
                    custom_constraint,
                    lb=np.zeros(3),
                    ub=np.zeros(3),
                    vars=np.concatenate([q[:,n-1], q[:,n]])
                )
        else:
            prog.AddConstraint(
                PositionConstraint(
                    plant,
                    plant.world_frame(),
                    [-np.inf, -np.inf, min_clearence],
                    [np.inf, np.inf, np.inf],
                    foot_frame[i],
                    p_FootFootTip,
                    context[n]
                ),
                vars=q[:, n]
            )


### 12. Periodicity Constraint
pose in the first time moment should be same the last one

In [34]:
# Everything except body_x is periodic
q_selector = PositionView([True] * n_q)
q_selector.body_x = False
prog.AddLinearConstraint(eq(q[q_selector, 0], q[q_selector, -1]))
prog.AddLinearConstraint(eq(v[:, 0], v[:, -1]))

## Objective function

In [35]:


# Joint costs weights
q_cost = PositionView([1] * n_q)
v_cost = VelocityView([1] * n_v)
q_cost.body_qw = 1
q_cost.body_qx = 1
q_cost.body_qy = 1
q_cost.body_qz = 1
q_cost.body_x = 0
q_cost.body_y = 0
q_cost.body_z = 0

q_cost.front_left_hip_x = 20
q_cost.front_right_hip_x = 20
q_cost.rear_left_hip_x = 20
q_cost.rear_right_hip_x= 20

q_cost.front_left_hip_y = 0
q_cost.front_right_hip_y = 0
q_cost.rear_left_hip_y = 0
q_cost.rear_right_hip_y= 0

q_cost.front_left_knee = 0
q_cost.front_right_knee = 0
q_cost.rear_left_knee = 0
q_cost.rear_right_knee = 0

v_cost.body_wx = 0
v_cost.body_wy = 0
v_cost.body_wz = 0
v_cost.body_vx = 0
v_cost.body_vy = 0
v_cost.body_vz = 0


# desired posture
q_desired = plant.GetDefaultPositions()
v_desired = [0] * n_v
for n in range(N):
    # Running costs:
    prog.AddQuadraticErrorCost(
            np.diag(q_cost), 
            q_desired, 
            q[:, n]
        )
    prog.AddQuadraticErrorCost(
            np.diag(v_cost), 
            v_desired, 
            v[:, n]
        )

## Initial Condition
setting quaternions to zero make the prog stuck at zero. The rest is just initialized to default pose:

In [36]:
for n in range(N):
    prog.SetInitialGuess(q[:, n], q_desired)

## Solve

In [37]:
snopt = SnoptSolver().solver_id()
prog.SetSolverOption(snopt, "Iterations Limits", 1e6)
prog.SetSolverOption(snopt, "Major Iterations Limit", 1e5)
prog.SetSolverOption(snopt, "Major Feasibility Tolerance", 5e-6)
prog.SetSolverOption(snopt, "Major Optimality Tolerance", 1e-4)
prog.SetSolverOption(snopt, "Superbasics limit", 3000)
prog.SetSolverOption(snopt, "Linesearch tolerance", 0.9)
# prog.SetSolverOption(snopt, 'Print file', 'snopt.out')


result = Solve(prog)
print(result.get_solver_id().name())


SNOPT


In [38]:
print(result.is_success())  # We expect this to be false if iterations are limited.

True


# Planned Play-back

In [39]:
 # Animate trajectory
context = diagram.CreateDefaultContext()
plant_context = plant.GetMyContextFromRoot(context)
simulator.reset_context(context)
t_sol = np.cumsum(np.concatenate(([0], result.GetSolution(h))))
q_sol = PiecewisePolynomial.FirstOrderHold(t_sol, result.GetSolution(q))
meshcat.StartRecording()
num_strides = 10
t0 = t_sol[0]
tf = t_sol[-1]
T = tf * num_strides
for t in np.hstack((np.arange(t0, T, 1.0 / 32.0), T)):
    context.SetTime(t)
    stride = (t - t0) // (tf - t0)
    ts = (t - t0) % (tf - t0)
    qt = PositionView(q_sol.value(ts))

    qt.body_x += stride * stride_length
    plant.SetPositions(plant_context, qt[:])
    diagram.ForcedPublish(context)

meshcat.StopRecording()
meshcat.PublishRecording()